In [ ]:
X = []
Z = []
IMG_SIZE = 150
FLOWER_DAISY_DIR = 'path/to/flower/daisy'
FLOWER_TULIP_DIR = 'path/to/flower/tulip'
FLOWER_SUNFLOWER_DIR = 'path/to/flower/tulip'
FLOWER_DANDY_DIR = 'path/to/flower/dandy'
FLOWER_ROSE_DIR = 'path/to/flower/rose'
def make_train_data(flower_type,DIR):
    flower_images = os.listdir(DIR)
    for i in range(len(flower_images)):
        img = Image.open(os.path.join(DIR,flower_images[i]))
        img = img.resize((IMG_SIZE, IMG_SIZE))
        data = np.array(img) / 255.0 - 0.5
        # data = (np.transpose(data,(1,2,0)))
        X.append(data)
        Z.append(flower_type)


In [ ]:

# Encode Y variable from categorical Z
from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()
y = labelencoder.fit_transform(Z)



In [ ]:
# Split x and y into train and test sets in the ratio 75/25
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split

x_train, y_train, x_test, y_test = train_test_split(X,y)
print("Train set:")
print(f"X shape: {x_train.shape}, y shape: {y_train.shape}")
print("\nTest set:")
print(f"X shape: {x_test.shape}, y shape: {y_test.shape}\n")

# Implementing Transfer Learning in Tensorflow

In [ ]:
# Implement transfer learning in tensorflow

from keras.applications import ResNet50
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf
# Load the pre-trained model without the top (classification) layer
base_model = ResNet50(weights='imagenet', include_top=False)
print("Base model loaded")

# Freeze the pre-trained layers so that they are not trainable
for layer in base_model.layers:
    layer.trainable = False


In [ ]:

# Add custom layers for classification on top of the ResNet50 base
x = base_model.output
x = GlobalAveragePooling2D()(x)  # use a global average pooling layer
x = Dense(1024, activation='relu')(x)   # add a fully connected layer


In [ ]:

# Create the final model
final_model = Model(inputs=base_model.input, outputs=x)

# Print summary of the new model
final_model.summary()


## Training Tensorflow Model

In [ ]:
num_epochs = 20
batch_size = 32
# Train the model
history = model.fit(x_train, y_train, epochs= num_epochs, batch_size=batch_size)


# Implementing Transfer Learning in PyTorch

In [ ]:
# Implement transfer learning in pytorch

import torch
from torch import nn
from torchvision.models import resnet5

# Load a pre-trained model (here, ResNet)
model = resnet50(pretrained=True)

print("Model loaded")

# Print the number of input features
input_features = model.fc.in_features
print("Number of input features: ", input_features)


In [ ]:

# Remove the last fully connected layer and print the new number of input features 
model.fc = torch.nn.Identity()
output_features = model.fc.in_features
print("Number of output features: ", output_features)


In [ ]:

# Freeze the parameters of the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Print the modified resnet50 architecture
print(model)


In [ ]:

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters())

## Train PyTorch Model

In [ ]:
# Training loop
from torch.utils.data import Dataset, DataLoader
class TrainDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        return image, label
# Create a data loader for training
train_dataset = TrainDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Update model parameters
        optimizer.step()

## PyTorch Model Evaluation

In [ ]:
# Train the model on the flower dataset
# Validation loop
with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for inputs, targets in val_loader:
        # Forward pass
        outputs = model(inputs)
        
        # Compute predictions
        _, predicted = torch.max(outputs, 1)
        
        # Compute accuracy
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)

    accuracy = total_correct / total_samples
    print(f'Validation Accuracy: {accuracy}')
